In [1]:
from __future__ import annotations
from dataclasses import dataclass, field
import os, sys
from typing import Any, Iterable, Literal
import hashlib
from limes_x.utils import KeyGenerator

class Namespace:
    def __init__(self) -> None:
        self.node_signatures: dict[str, str] = {}
        self._keygen = KeyGenerator()
        self._keys: set[str] = set()

    def NewKey(self):
        return self._keygen.GenerateUID(blacklist=self._keys)

class Hashable:
    def __init__(self, ns: Namespace) -> None:
        self.namespace = ns
        self.key = ns.NewKey()
        self.hash = int(hashlib.md5(self.key.encode("latin1")).hexdigest(), 16)

    def __hash__(self) -> int:
        return self.hash
    
    def __eq__(self, __value: object) -> bool:
        K = "hash"
        return hasattr(__value, K) and self.hash == getattr(__value, K)

class Node(Hashable):
    def __init__(
        self,
        ns: Namespace,
        properties: set[str],
        parents: set[Node],
    ) -> None:
        super().__init__(ns)
        self.namespace = ns
        self.properties = properties
        self.parents = parents

    def __str__(self) -> str:
        return f"<{self.key}:{','.join(self.properties)}>"

    def __repr__(self) -> str:
        return f"{self}"
    
    # # x == y if x is a "subset" of y
    # # that is, x has at least all features of y
    # def __eq__(self, __value: object) -> bool:
    #     if not isinstance(__value, Node): return False
    #     # if not __value.properties.issubset(self.properties): return False
    #     for p in __value.properties:
    #         if p not in self.properties: return False
    #     for p in __value.parents:
    #         if all(p != op for op in self.parents): return False
    #     return True
    
    def IsA(self, other: Node) -> bool:
        if not other.properties.issubset(self.properties): return False
        # if not other.parents.issubset(self.parents): return False
        return True

    def Signature(self):
        cache = self.namespace.node_signatures
        if self.key not in cache:
            props = "".join(sorted(self.properties))
            parents = "".join(sorted([p.Signature() for p in self.parents]))
            sig = props+parents
            cache[self.key] = sig
        return cache[self.key]

    def MatchesMemberOf(self, collection: Iterable[Node]):
        return any(self == m for m in collection)

class Dependency(Node):
    def __init__(self, namespace: Namespace, properties: set[str], parents: set[Node]) -> None:
        super().__init__(namespace, properties, parents)

class Endpoint(Node):
    def __init__(self, namespace: Namespace, properties: set[str], parents: set[Node]=set()) -> None:
        super().__init__(namespace, properties, parents)

class Transform(Hashable):
    def __init__(self, ns: Namespace) -> None:
        super().__init__(ns)
        self.requires: list[Dependency] = []
        self.produces: list[Dependency] = []
        self._ns = ns
        self._input_group_map: dict[int, list[Dependency]] = {}
        self._key = ns.NewKey()

    def __str__(self) -> str:
        def _props(d: Dependency):
            return "{"+"-".join(d.properties)+"}"
        return f"<{','.join(_props(r) for r in self.requires)}->{','.join(_props(p) for p in self.produces)}>"

    def __repr__(self): return f"{self}"

    def AddRequirement(self, properties: Iterable[str], parents: set[Dependency]=set()):
        return self._add_dependency(self.requires, properties, parents)

    def AddProduct(self, properties: Iterable[str], parents: set[Dependency]=set()):
        return self._add_dependency(self.produces, properties, parents)

    def _add_dependency(self, destination: list[Dependency], properties: Iterable[str], parents: set[Dependency]=set()):
        _parents: Any = parents
        _dep = Dependency(properties=set(properties), parents=_parents, namespace=self._ns)
        assert not any(e.IsA(_dep) for e in destination), f"prev. dep ⊆ new dep"
        assert not any(_dep.IsA(e) for e in destination), f"new dep ⊆ prev. dep "
        destination.append(_dep)
        if destination == self.requires:
            i = len(self.requires)-1
            for p in _parents:
                assert p in self.requires, f"{p} not added as a requirement"
            self._input_group_map[i] = self._input_group_map.get(i, [])+list(_parents)
        return _dep

    def _sig(self, endpoints: Iterable[Endpoint]):
        return "".join(e.key for e in endpoints)

    def Apply(self, have: Iterable[Endpoint], blacklist: set[str]):
        matches: list[list[Endpoint]] = []
        for req in self.requires:
            _m = [m for m in have if m.IsA(req)]
            if len(_m) == 0: return []
            matches.append(_m)

        # can reduce exponential trial here by enforcning the input groups first
        def _possible_configs(i: int, choosen: list[Endpoint]) -> list[list[Endpoint]]:
            if i >= len(self.requires): return [choosen]
            candidates = matches[i]
            parents = self._input_group_map.get(i, [])
            # print(parents, candidates, choosen)
            if len(parents) > 0:
                for prototype in parents:
                    # parent must be in choosen, since it must have been added
                    # as a req. before being used as a parent
                    parent: None|Endpoint = None
                    for p in choosen:
                        if p.IsA(prototype): parent = p; break
                    if parent is None: return []
                    candidates = [c for c in candidates if parent in c.parents]
            configs = []
            for c in candidates:
                configs += _possible_configs(i+1, choosen+[c])
            return configs
        configs = _possible_configs(0, [])

        # todo: next optimization is DFS, with saved subplans

        applications: list[Application] = []
        for input_set in configs:
            sis = set(input_set)
            sig = self._sig(input_set)
            if sig in blacklist: continue
            _parents = sis|{p for g in [e.parents for e in input_set] for p in g}
            produced = [
                Endpoint(
                    namespace=self._ns,
                    properties=out.properties,
                    parents=_parents
                )
            for out in self.produces]
            applications.append(Application(self, sis, produced, sig))
        return applications

@dataclass
class Application:
    transform: Transform
    used: set[Endpoint]
    produced: list[Endpoint]
    signature: str

@dataclass
class Result:
    solution: list[Application]
    message: str = ""
    evidence: Any = None
    steps: int = 0
    
def Solve(given: Iterable[Endpoint], targets: Iterable[Endpoint], transforms: Iterable[Transform]):
    if all(any(t.IsA(g) for g in given) for t in targets): return Result([], "given all targets")

    @dataclass
    class State:
        requirements: list[Endpoint]
        have: list[Endpoint]
        usage_signatures: dict[str, set[str]]
        plan: list[Application]
        last_tr_i: int

    transforms = list(transforms)
    solution_transforms: set[int] = set()
    for i, tr in enumerate(transforms):
        for p in tr.produces:
            if not any(p.IsA(t) for t in targets): continue
            solution_transforms.add(i)
    todo = [State(
        requirements = list(targets),
        have = list(given),
        plan = [],
        last_tr_i = 0,
        usage_signatures={},
    )]
    _steps = 0
    MAXS = 9999
    while len(todo)>0:
        _steps += 1
        if _steps > MAXS: return Result([], f"step limit exceeded", steps=_steps)
        _s = todo.pop()
        if len(_s.requirements) == 0: return Result(_s.plan, steps = _steps)
        
        tri = _s.last_tr_i+1
        if tri>len(transforms): tri = 0
        trs = transforms[tri:]+transforms[:tri]
        # print(trs)
        for tr in trs:

            for app in tr.Apply(_s.have, _s.usage_signatures.get(tr.key, set())):
                new_reqs = _s.requirements
                if tri in solution_transforms:
                    new_reqs = new_reqs.copy()
                    for i in range(len(new_reqs)-1, -1, -1):
                        req = new_reqs[i]
                        if any(p.IsA(req) for p in app.produced): new_reqs.remove(req)
                # print(len(_s.plan), tri, app)
                sigs = _s.usage_signatures.copy()
                sigs[tr.key] = sigs.get(tr.key, set())|{app.signature}
                todo.append(State(
                    requirements = new_reqs,
                    have = _s.have+app.produced,
                    last_tr_i = tri,
                    plan = _s.plan+[app],
                    usage_signatures = sigs,
                ))
    return Result([], f"ran out of things to try", steps = _steps)

# x, steps = Solve([asm, bin], [sum_asm, sum_bin], [anner, taxer, sumer])
# x, steps

# x, steps = Solve([asm, bin]+bs, [sum_asm, sum_bin]+ss, [anner, taxer, sumer])
# x, steps

NS = Namespace()
def _set(s: str):
    return set(s.split(", "))

anner = Transform(NS)
anner.AddRequirement(_set("annable"))
anner.AddProduct(_set("ann"))

taxer = Transform(NS)
taxer.AddRequirement(_set("taxable"))
taxer.AddProduct(_set("tax"))

sumer = Transform(NS)
d_parent = sumer.AddRequirement(_set("annable, taxable"))
d_ann = sumer.AddRequirement(_set("ann"), {d_parent})
d_tax = sumer.AddRequirement(_set("tax"), {d_parent})
sumer.AddProduct(_set("sum"))

N = 100

bs = [Endpoint(NS, _set(f"{i+1}, annable, taxable")) for i in range(N)]
ss = [Endpoint(NS, _set("sum"), {e}) for e in bs]
tr = [anner, taxer, sumer]
# %prun Solve(bs, ss, [anner, taxer, sumer])
# r = Solve(bs, ss, [anner, taxer, sumer])
# r

test_have = []
for b in bs[:N]:
    test_have.append(b)
    test_have.append(Endpoint(NS, _set("ann"), {b}))
    test_have.append(Endpoint(NS, _set("tax"), {b}))

# sumer.Apply(test_have)
print("Start")
%prun r = Solve(bs, ss, tr)
f"input size [{N}], states checked [{r.steps}]"

Start
 

'input size [100], states checked [304]'

         9482072 function calls (9365620 primitive calls) in 2.473 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    42974    0.363    0.000    0.479    0.000 3144352985.py:143(<listcomp>)
        1    0.316    0.316    2.457    2.457 3144352985.py:182(Solve)
    27725    0.196    0.000    0.276    0.000 fromnumeric.py:69(_wrapreduction)
  1120820    0.182    0.000    0.235    0.000 3144352985.py:59(IsA)
  1394800    0.163    0.000    0.313    0.000 3144352985.py:226(<genexpr>)
   697404    0.152    0.000    0.465    0.000 {built-in method builtins.any}
  2502822    0.126    0.000    0.126    0.000 3144352985.py:23(__hash__)
117355/907    0.113    0.000    0.613    0.001 3144352985.py:130(_possible_configs)
      909    0.091    0.000    1.509    0.002 3144352985.py:122(Apply)
   366300    0.080    0.000    0.112    0.000 3144352985.py:26(__eq__)
    27725    0.059    0.000    0.388    0.000 {method 'randint' of 'numpy.ra

In [58]:
len(r.solution), r.steps

(10, 16)

In [70]:
r.solution

[Application(transform=<{annable}->{ann}>, used={<ryIimv2Ja9KS:annable,4,taxable>}, produced=[<4NRzojmhcJlh:ann>], signature='ryIimv2Ja9KS'),
 Application(transform=<{taxable}->{tax}>, used={<ryIimv2Ja9KS:annable,4,taxable>}, produced=[<LlxShHV0KfYo:tax>], signature='ryIimv2Ja9KS'),
 Application(transform=<{annable-taxable},{ann},{tax}->{sum}>, used={<ryIimv2Ja9KS:annable,4,taxable>, <4NRzojmhcJlh:ann>, <LlxShHV0KfYo:tax>}, produced=[<kGC0ewpNoyY9:sum>], signature='ryIimv2Ja9KS4NRzojmhcJlhLlxShHV0KfYo'),
 Application(transform=<{taxable}->{tax}>, used={<d3UN0uBb10oO:3,taxable,annable>}, produced=[<MjLKKBe8ut3M:tax>], signature='d3UN0uBb10oO'),
 Application(transform=<{annable}->{ann}>, used={<d3UN0uBb10oO:3,taxable,annable>}, produced=[<4IcUcAw2ka73:ann>], signature='d3UN0uBb10oO'),
 Application(transform=<{taxable}->{tax}>, used={<ESD4GQHUimAV:2,taxable,annable>}, produced=[<9Ev3sT9oLCG9:tax>], signature='ESD4GQHUimAV'),
 Application(transform=<{annable-taxable},{ann},{tax}->{sum}>, u

In [59]:
# # test all given
# r = Solve(bs, bs, [anner, taxer, sumer])

In [60]:
# def Solve(given: Iterable[Endpoint], targets: Iterable[Endpoint], transforms: Iterable[Transform]):
#     todo: list[tuple[set[Endpoint], list[Application]]] = [
#         (set(given), [])
#     ]
#     def _signature(state: Iterable[Endpoint]):
#         return ",".join(e.Signature() for e in state)

#     steps = 0
#     seen = set()
#     while len(todo) > 0:
#         steps += 1
#         _have, _path = todo.pop(0)
#         if all(any(e == t for e in _have) for t in targets): return _path, steps
#         for tr in transforms:
#             applications = tr.Apply(_have, _path)
#             for appl in applications:
#                 new_have = _have | set(appl.produced)
#                 # sig = _signature(new_have)
#                 # if sig in seen: continue
#                 # seen.add(sig)
#                 todo.append((new_have, _path+[appl]))
#     return False, steps

In [61]:
# _start = Endpoint(namespace=NS, properties=_settify("primeable"))
# _target = Endpoint(namespace=NS, properties=_settify("starred"), parents={_start})

# primer = Transform(NS)
# primer.AddDependency(
#     "req", _settify("primeable"),
# )
# primer.AddDependency(
#     "prod", _settify("primed"),
# )

# starer = Transform(NS)
# starer.AddDependency(
#     "req", _settify("primed"),
# )
# starer.AddDependency(
#     "prod", _settify("starred"),
# )

# have = {_start}
# targets = {_target}

# def Solve(have: Iterable[Endpoint], targets: Iterable[Endpoint]):
#     todo = [
#         (set(have), [])
#     ]
#     while len(todo)

# xs = primer.Apply([_start])
# ys = starer.Apply(xs)

# for x in ys:
#     print(x == _target, _target == x)
#     print(x.parents, x.properties)
#     print(_target.parents, _target.properties)

In [62]:
# def Solve(given: Iterable[Endpoint], targets: Iterable[Endpoint], transforms: Iterable[Transform]):
#     if all(any(t.IsA(g) for g in given) for t in targets): return Result([], "given all targets")

#     @dataclass
#     class State:
#         targets: set[Endpoint]
#         usage_signatures: set[str]
#         plan: list[Application]

#     prod_map: dict[str, set[Transform]] = {}
#     for tr in transforms:
#         for prod in tr.produces:
#             for prop in prod.properties:
#                 prod_map[prop] = prod_map.get(prop, set()) | {tr}

#     # sub_solutions: dict[Endpoint, list[Transform]] = {}
#     def _solve_target(target: Node):
#         # if can't produce a property, can't produce target
#         if any(p not in prod_map for p in target.properties): return []
#         # get transforms that can create all properties
#         candidates: None|set[Transform] = None
#         for p in target.properties:
#             if candidates is None: candidates = prod_map[p]
#             else: candidates = candidates ^ prod_map[p]
#         if candidates is None or len(candidates) == 0: return []
#         # ensure transforms can create target. properties are no same product
#         valid_transforms: list[Transform] = []
#         for tr in candidates:
#             for prod in tr.produces:
#                 # print(prod.properties, target.properties)
#                 # print(prod.parents, target.parents)
#                 if not prod.IsA(target): continue
#                 valid_transforms.append(tr)
#                 break
#         return valid_transforms

#     given_props = {p for g in [g.properties for g in given] for p in g}
#     def _in_given(n: Node):
#         if not n.properties.issubset(given_props): return False
#         useable = [g for g in given if g.IsA(n)]
#         if len(useable) == 0: return False
#         return useable
    
#     def signature():
#         pass

#     s_given: set[Endpoint] = set(given)
#     s_targets: set[Endpoint] = set(targets)
#     todo: list[State] = [State(
#         targets=s_targets-s_given,
#         usage_signatures=set(),
#         plan=[],
#     )]
#     while len(todo)>0:
#         print(">")
#         _s = todo.pop()
#         if len(_s.targets) == 0: return Result(_s.plan) # solved!
        
#         valid_transforms = {tr for g in [_solve_target(t) for t in _s.targets] for tr in g}
#         if len(valid_transforms) == 0: return Result(_s.plan, "no valid transforms for", _s.targets)
#         print(valid_transforms, _s.targets)
#         for tr in valid_transforms:
#             reqs, pending = [], set()
#             for r in tr.requires:
#                 useable_givens = _in_given(r)
#                 _continue = False
#                 if useable_givens:
#                     for g in useable_givens:
#                         if g in used_givens: continue
#                         reqs.append(g)
#                         used_givens.add(g)
#                         _continue=True; break
#                 if _continue: continue
#                 n = Endpoint

#                 reqs.append(r)
#                 pending.add(r)
        
#             produced = set()
#             for p in tr.produces:
#                 for t in _s.targets:
#                     if t in produced: continue # comparison using exact hash
#                     if p.IsA(t):
#                         produced.add(t)
#                         break
#             todo.append(State(
#                 targets=_s.targets-produced|pending,
#                 usage_signatures=_s.usage_signatures.copy(),
#                 plan=_s.plan+[Application(tr, reqs, produced)],
#             ))

#     return Result([], "todo exhausted")

In [63]:
  
# NS = Namespace()
# anner = Transform("anner", NS)
# anner.AddDependency(
#     "req", "a_in",
#     "annable".split(", "),
# )
# anner.AddDependency(
#     "prod", "a_out",
#     "ann".split(", "),
# )

# taxer = Transform("taxer", NS)
# taxer.AddDependency(
#     "req", "t_in",
#     "taxable".split(", "),
# )
# taxer.AddDependency(
#     "prod", "t_out",
#     "tax".split(", "),
# )

# sumer = Transform("sumer", NS)
# sumer.AddDependency(
#     "req", "s_in_ann",
#     "ann".split(", "),
# )
# sumer.AddDependency(
#     "req", "s_in_tax",
#     "tax".split(", "),
# )
# sumer.AddDependency(
#     "prod", "s_out",
#     "sum".split(", "),
# )

# in_asm = Endpoint.New(NS, "in_asm", "asm, annable, taxable", have=True)
# in_bin = Endpoint.New(NS, "in_bin", "bin, annable, taxable", have=True)

# for tr in [anner, taxer, sumer]:
#     pass

In [64]:
# from __future__ import annotations
# from dataclasses import dataclass, field
# import os, sys
# from typing import Any, Iterable, Literal
# import networkx as nx
# import hashlib

# class Namespace:
#     def __init__(self) -> None:
#         self.node_hashes: dict[str, int] = {}
#         self.properties: dict[str, Property] = {}

#     def GetProperty(self, key: str):
#         if key not in self.properties:
#             new = Property(self, key)
#             new.back_links = set()
#             self.properties[key] = new
#         return self.properties[key]

# @dataclass
# class Node:
#     namespace: Namespace
#     key: str

#     def __hash__(self) -> int:
#         node_hashes = self.namespace.node_hashes
#         if self.key not in node_hashes:
#             node_hashes[self.key] = int(hashlib.md5(self.key.encode("latin1")).hexdigest(), 16)
#         return node_hashes[self.key]
    
#     def __eq__(self, __value: object) -> bool:
#         if not isinstance(__value, type(self)): return False
#         return self.key == __value.key

# class Linkable:
#     back_links: set[HasLinks]

# @dataclass
# class Haveable:
#     have: bool

# @dataclass
# class HasLinks:
#     links: set[Linkable]

#     def Enforce_backlinks(self):
#         for o in self.links:
#             o.back_links.add(self)

#     def Link(self, o: Linkable):
#         self.links.add(o)
#         o.back_links.add(self)

#     def Clear(self):
#         for o in self.links:
#             o.back_links.remove(self)
#         self.links.clear()

#     def Matches(self, other: HasLinks):
#         return all(l in other.links for l in self.links)

# @dataclass
# class Property(Node, Linkable):
#     def __hash__(self) -> int: return Node.__hash__(self)
#     def __eq__(self, __value: object) -> bool: return Node.__eq__(self, __value)
    
# @dataclass
# class Template(Node, HasLinks):
#     pass

# @dataclass
# class Dependency(Node, HasLinks, Haveable):
#     template: Template
#     def __hash__(self) -> int: return Node.__hash__(self)
#     def __eq__(self, __value: object) -> bool: return Node.__eq__(self, __value)

#     def Reset(self):
#         self.Clear()
#         self.links = self.template.links.copy()
#         self.Enforce_backlinks()

# @dataclass
# class Endpoint(Node, HasLinks, Linkable, Haveable):
#     def __hash__(self) -> int: return Node.__hash__(self)
#     def __eq__(self, __value: object) -> bool: return Node.__eq__(self, __value)

#     @classmethod
#     def New(cls, ns: Namespace, key: str, properties: Iterable[str], parents: set[Linkable]=set(), have=False):
#         _links: set[Linkable] = {ns.GetProperty(p) for p in properties}
#         _links = _links.union(parents)
#         return Endpoint(
#             key = key, links = _links,
#             have = have, namespace=ns,
#         )

# class Transform:
#     def __init__(self, name: str, namespace: Namespace) -> None:
#         self.requires: set[Dependency] = set()
#         self.produces: set[Dependency] = set()
#         self.raw: bool = True
#         self.name = name
#         self._ns = namespace

#     def __repr__(self) -> str:
#         return f"Tr:{self.name}"

#     def AddDependency(self, role: Literal["req"]|Literal["prod"], key: str, properties: Iterable[str], parents: set[Linkable]=set()):
#         _links: set[Linkable] = {self._ns.GetProperty(p) for p in properties}
#         _links = _links.union(parents)
#         _template = Template(key=f"T-{key}", links =_links, namespace=self._ns)
#         _dep = Dependency(key=key, links=_links, template=_template, have=False, namespace=self._ns)
#         _dep.Enforce_backlinks() # should be in init, but @_dep is dataclass!

#         if role == "req":
#             assert _dep not in self.produces
#             self.requires.add(_dep)
#         else:
#             assert _dep not in self.requires
#             self.produces.add(_dep)

#     def Reset(self):
#         self.raw = True
#         for d in self.requires | self.produces:
#             d.Reset()
        
# NS = Namespace()
# anner = Transform("anner", NS)
# anner.AddDependency(
#     "req", "a_in",
#     "annable".split(", "),
# )
# anner.AddDependency(
#     "prod", "a_out",
#     "ann".split(", "),
# )

# taxer = Transform("taxer", NS)
# taxer.AddDependency(
#     "req", "t_in",
#     "taxable".split(", "),
# )
# taxer.AddDependency(
#     "prod", "t_out",
#     "tax".split(", "),
# )

# sumer = Transform("sumer", NS)
# sumer.AddDependency(
#     "req", "s_in_ann",
#     "ann".split(", "),
# )
# sumer.AddDependency(
#     "req", "s_in_tax",
#     "tax".split(", "),
# )
# sumer.AddDependency(
#     "prod", "s_out",
#     "sum".split(", "),
# )

# in_asm = Endpoint.New(NS, "in_asm", "asm, annable, taxable", have=True)
# in_bin = Endpoint.New(NS, "in_bin", "bin, annable, taxable", have=True)

# for tr in [anner, taxer, sumer]:
#     pass

In [65]:
# from __future__ import annotations
# import os, sys
# import asyncio
# from typing import Iterable, Callable, Any
# from pathlib import Path

# from limes_x.solver import DependencySolver, Plan, Dependency
# from limes_x.persistence import ProjectState, Instance
# from limes_x.compute_module import ComputeModule

# mpath = Path("./test_solver/")
# modules = [
#     ComputeModule(mpath.joinpath(d)) for d in os.listdir(mpath)
# ]
# print(modules)

# given = [
#     ("a", "./test_data/a1"),
#     ("a", "./test_data/a2"),
#     ("b", "./test_data/b1"),
# ]

# prj_path = "./cache/man_test01/"
# state = ProjectState(prj_path, on_exist="overwrite")
# for dtype, val in given:
#     state.RegisterInstance(Instance.Str(dtype, val))
# for m in modules:
#     state.RegisterInstance(Instance.ComputeModule(m))

# deps = []
# for k, inst in state._instances.items():
#     if not inst.IsPyType(ComputeModule): continue
#     deps.append(Dependency(inst.val.requires, inst.val.produces, k))

# solver = DependencySolver(deps)
# # plan = solver.Solve({"a"}, {"reuse", "linear", "branched"})
# plan = solver.Solve({"a"}, {"branched"})
# assert plan != False
# [state.GetInstance(m.ref_key) for m in plan]

In [66]:
# def make_dependency(module: ComputeModule):
#     return Dependency(module.requires, module.produces, module)

# modules = Path("./test_solver/")
# solver = Plan([
#     make_dependency(ComputeModule(p))
# for p in [
#     modules.joinpath(p) for p in os.listdir(modules)
# ]])
# plan = solver.Solve({"a"}, {"reuse", "linear", "branched"})
# plan

In [67]:
# from limes_x.compute_module import ComputeModule

# a = ComputeModule("./test_modules/copy/")
# b = ComputeModule("./test_modules/copy2/")

# a.requires, b.requires

In [68]:
# state = ProjectState("./cache/test_persist")
# ok = Instance("asdf", 1)
# ov = Instance("s", 2)
# state._lineage[ok] = [ov]
# state.Save()

# s2 = ProjectState.Load("./cache/test_persist")
# for k, v in s2._lineage.items():
#     _te = k.type, k.value, ok == k, [(i.type, i.value, i == ov) for i in v]
#     print(_te)

# ok._id

In [69]:
# test = []
# for i in range(100000):
#     x = Instance("asdf", ["x"*150, "y"*150])
#     # x = 1
#     test.append(x)